# Feast Notebook Quickstart

## Install feast package

### From git
Consider this option if you need to use features under development not yet released.

In [1]:
!git clone https://github.com/feast-dev/feast.git

Cloning into 'feast'...
remote: Enumerating objects: 66378, done.
remote: Counting objects: 100% (1139/1139), done.
remote: Compressing objects: 100% (622/622), done.
remote: Total 66378 (delta 628), reused 828 (delta 435), pack-reused 65239 (from 1)
Receiving objects: 100% (66378/66378), 88.10 MiB | 29.56 MiB/s, done.
Resolving deltas: 100% (41982/41982), done.


In [4]:
%pip install -q -e "feast[dev,postgres]"


[notice] A new release of pip available: 22.2.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


**Note**: Additional dependencies might be needed according to the actual feature store components. Recommendation is to install them using the extras dependencies
defined in `setup.py` to guarantee consistency against the validated versions.

### From python package repository
Consider this option if you need to use features already released.

In [7]:
# Use this if you want a stable version
# %pip install -q "feast[postgres]"

## Validate the Notebook file system
Verify that the client ConfigMap and the TLS secrets have been properly mounted.

In [8]:
!find /feast

/feast
/feast/feature_repository
/feast/feature_repository/..2024_12_05_10_02_40.4029380524
/feast/feature_repository/..2024_12_05_10_02_40.4029380524/feature_store.yaml
/feast/feature_repository/..data
/feast/feature_repository/feature_store.yaml


In [9]:
!cat /feast/feature_repository/feature_store.yaml

project: sample
provider: local
offline_store:
    host: feast-example-offline.feast.svc.cluster.local
    type: remote
    port: 80
online_store:
    path: https://feast-example-online.feast.svc.cluster.local:443
    type: remote
    cert: /tls/online/tls.crt
registry:
    path: feast-example-registry.feast.svc.cluster.local:443
    registry_type: remote
    cert: /tls/registry/tls.crt
auth:
    type: no_auth
entity_key_serialization_version: 3


In [10]:
!find /tls

/tls
/tls/online
/tls/online/tls.key
/tls/online/tls.crt
/tls/online/..data
/tls/online/..2024_12_05_10_02_40.3439156357
/tls/online/..2024_12_05_10_02_40.3439156357/tls.key
/tls/online/..2024_12_05_10_02_40.3439156357/tls.crt
/tls/registry
/tls/registry/tls.key
/tls/registry/tls.crt
/tls/registry/..data
/tls/registry/..2024_12_05_10_02_40.1732177020
/tls/registry/..2024_12_05_10_02_40.1732177020/tls.key
/tls/registry/..2024_12_05_10_02_40.1732177020/tls.crt


## Validate communication using feast CLI
Run basic commands to verify the content of the  feature store.

In [11]:
%env FEATURE_REPO=/feast/feature_repository

env: FEATURE_REPO=/feast/feature_repository


In [12]:
!feast -c $FEATURE_REPO entities list 

NAME    DESCRIPTION    TYPE
driver                 ValueType.UNKNOWN


In [13]:
!feast -c $FEATURE_REPO feature-views list

NAME                         ENTITIES    TYPE
driver_hourly_stats_fresh    {'driver'}  FeatureView
driver_hourly_stats          {'driver'}  FeatureView
transformed_conv_rate_fresh  {'driver'}  OnDemandFeatureView
transformed_conv_rate        {'driver'}  OnDemandFeatureView


## Validate communication using feast modules
Use feast python modules to verify and manage the feature store.

In [14]:
import os

from feast.feature_store import FeatureStore

store = FeatureStore(os.environ['FEATURE_REPO'])
for data_source in store.registry.list_data_sources(store.project):
    print(data_source.name)


vals_to_add
driver_stats_push_source
driver_hourly_stats_source
